## Exemplo de scraping com formulários
***

In [1]:
import requests
from bs4 import BeautifulSoup
from http import HTTPStatus

In [2]:
url = "https://www.detran.mg.gov.br/veiculos/situacao-do-veiculo/emissao-de-boleto-de-multas"

In [3]:
session = requests.Session()

In [4]:
def get_vehicle_form_html():
    """
    Entra na página de formulário para a consulta das multas do veículo.
    """

    response = session.get(url)

    if response.status_code != HTTPStatus.OK:
        raise Exception(
            "O site do detran M.G está fora do ar.",
            status_code=HTTPStatus.BAD_REQUEST
        )

    return response.text

In [5]:
def submit_form(data):
    """
    Envia a submissão do formulario para pegar as multas
    do veículo.
    """

    form_url = url + "/-/exibir_lista_tipos_infracoes_multas/"

    header = {'Content-type': 'application/x-www-form-urlencoded'}

    response = session.post(form_url, data=data, headers=header)

    if response.status_code != HTTPStatus.OK:
        raise Exception(
            "O site do detran M.G está fora do ar.",
            status_code=HTTPStatus.BAD_REQUEST
        )

    return response.text

In [6]:
def get_tickets_list():
    """
    Pega o html com a lista de infrações.
    """

    ticket_url = url + "/-/exibir_lista_tipos_infracoes_multas/-/lista_tipo_infracoes_para_extrato/{0}/{1}/0/0/0/0/0/0/3/".format(
      "HFL5661", "00911263128"
    )

    response = session.get(ticket_url)

    if response.status_code != HTTPStatus.OK:
        raise Exception(
            "O site do detran M.G está fora do ar.",
            status_code=HTTPStatus.BAD_REQUEST
        )

    return response.text

In [7]:
html_string = get_vehicle_form_html()

In [8]:
data = {
    "data[EmitirExtratoMulta][placa]": "HFL5661",
    "data[EmitirExtratoMulta][renavam]": "00911263128"
}

In [9]:
html_string = submit_form(data)

In [10]:
html_string = get_tickets_list()

In [11]:
soup = BeautifulSoup(html_string, "html.parser")

In [12]:
tickets = []

In [13]:
def format_amount(amount):
    """
    Formata a string de saida do valor
    """

    string = amount.replace("R$ ", "").replace(",", ".")

    return float(string)

In [14]:
try:
    table = soup.find("tbody")
    for tr in table.find_all("tr"):
        tds = tr.find_all("td")
        tickets.append({
            "id": tds[1].string,
            "description": tds[2].string.capitalize(),
            "county": tds[3].string.capitalize(),
            "amount": format_amount(tds[4].string),
            "url": "https://www.detran.mg.gov.br" + tds[5].a['href']
        })
except Exception as error:
    raise Exception(
        "Ocorreu um problema ao pegar os dados do detran MG",
        status_code=HTTPStatus.BAD_REQUEST
    )

In [15]:
print(tickets)

[{'county': 'Rua  padre  augusto 183', 'url': 'https://www.detran.mg.gov.br/veiculos/situacao-do-veiculo/emissao-de-boleto-de-multas/-/emissao_extrato_multa/HFL5661/00911263128/248650/1/6129671/5320/', 'id': '6129671', 'amount': 53.2, 'description': 'Estacionar em desacordo com a regulamentacao - vaga de carga'}]
